In [1]:
using FFTW, Random, Test

Test data

In [2]:
Random.seed!(1234)
fftw_sample_input = rand(1024);

In [3]:
function test_fft(fft)
    @test FFTW.fft(fftw_sample_input) ≈ fft(fftw_sample_input)
end

test_fft (generic function with 1 method)

In [4]:
# function fft(dir, l)
#     n = length(l)
#     @assert ispow2(n)
#     if (isone(n))
#         l
#     else
#         l1 = [v[i] for i in 1 : 2 : n ]
#         l2 = [v[i] for i in 2 : 2 : n ]
#         y0 = fft(dir, l1)
#         y1 = fft(dir, l2)
#         merge(dir, y0, y1)
#     end
# end

In [5]:
# w(dir, n. j) = ?

In [6]:
# function merge(dir, l1, l2)
#     n1 = length(l1)
#     n2 = length(l2)
#     @assert n1 == n2
#     n = n1 + n2
#     let rec mg l1 l2 j =
#         match (l1, l2) with
#         (x::xs, y::ys) ->
#             let z1 = mult (w dir n j, y) in
#             let zx = add (x, z1) in
#             let zy = sub (x, z1) in
#             let (a,b) = (mg xs ys (j+1)) in
#             (zx::a, zy::b)
#         | _ -> ([], []) in
#     let (a,b) = mg l1 l2 0
#          in (a @ b)
# end

In [7]:
function naive_dft(x)
    N = length(x)
    k = reshape(0:(N-1), 1, :)
    n = 0:(N-1)
    M = @. exp(-2 * im * π * k * n / N)
    M * x
end

naive_dft (generic function with 1 method)

In [8]:
test_fft(naive_dft)

Test Passed

In [9]:
function naive_dft2(x)
    @show N = length(x)
    @show k = reshape(0:(N-1), 1, :)
    n = 0:(N-1)
    M = @. exp(-2 * im * π * k * n / N)
    M * x
end

naive_dft2 (generic function with 1 method)

In [10]:
naive_dft2([10, 20, 30, 40])

N = length(x) = 4
k = reshape(0:N - 1, 1, :) = [0 1 2 3]


4-element Vector{ComplexF64}:
               100.0 + 0.0im
 -20.000000000000007 + 19.999999999999996im
               -20.0 - 9.797174393178823e-15im
 -19.999999999999982 - 20.00000000000001im

In [11]:
function dft_rec(a)
    n = length(a)
    @assert ispow2(n)
    isone(n) && return a
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1 = dft_rec(a1)
    y2 = dft_rec(a2)

    wk = [w^(k - 1) for k in 1:n2]
    z1 = [y1[k] + wk[k] * y2[k] for k in 1:n2]
    z2 = [y1[k] - wk[k] * y2[k] for k in 1:n2]
    return vcat(z1, z2)
end

dft_rec (generic function with 1 method)

In [12]:
dft_rec([10, 20, 30, 40])

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [13]:
test_fft(dft_rec)

Test Passed

In [14]:
function dft_rec2(a::Vector{T}, b::Vector{Complex{T}}) where {T}
    n = length(a)
    @assert ispow2(n)
    isone(n) && (b[1] = a[1]; return a)
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    a1 = [a[k] for k in 1:2:n]
    a2 = [a[k] for k in 2:2:n]
    y1 = Vector{Complex{T}}(undef, n2)
    y2 = Vector{Complex{T}}(undef, n2)
    dft_rec2(a1, y1)
    dft_rec2(a2, y2)
    wk = [w^(k - 1) for k in 1:n2]
    for k in 1:n2
        wky2 = wk[k] * y2[k]
        b[k]    = y1[k] + wky2
        b[k+n2] = y1[k] - wky2
    end
    return
end

dft_rec2 (generic function with 1 method)

In [15]:
function dft_rec2(a::Vector{T}) where {T}
    b = Vector{Complex{T}}(undef, length(a))
    dft_rec2(a, b)
    return b
end

dft_rec2 (generic function with 2 methods)

In [16]:
dft_rec2([10.0, 20.0, 30.0, 40.0])

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [17]:
test_fft(dft_rec2)

Test Passed

In [18]:
function dft_rec3(a::Vector{T}, p, b::Vector{Complex{T}}) where {T}
    n = length(p)
    isone(n) && (b[1] = a[p[1]]; return a)
    n2 = n ÷ 2
    w = exp(-2 * pi * im / n)

    p1 = [p[k] for k in 1:2:n]
    p2 = [p[k] for k in 2:2:n]
    y1 = Vector{Complex{T}}(undef, n2)
    y2 = Vector{Complex{T}}(undef, n2)
    dft_rec3(a, p1, y1)
    dft_rec3(a, p2, y2)
    wk = [w^(k - 1) for k in 1:n2]
    for k in 1:n2
        wky2 = wk[k] * y2[k]
        b[k]    = y1[k] + wky2
        b[k+n2] = y1[k] - wky2
    end
    return
end

dft_rec3 (generic function with 1 method)

In [19]:
function dft_rec3(a::Vector{T}) where {T}
    n = length(a)
    @assert ispow2(n)
    p = [k for k in 1:n]
    b = Vector{Complex{T}}(undef, length(a))
    dft_rec3(a, p, b)
    return b
end

dft_rec3 (generic function with 2 methods)

In [20]:
dft_rec3([10.0, 20.0, 30.0, 40.0])

4-element Vector{ComplexF64}:
 100.0 + 0.0im
 -20.0 + 20.0im
 -20.0 + 0.0im
 -20.0 - 20.0im

In [21]:
test_fft(dft_rec3)

Test Passed